## 1) Feature Generation
학습 기간을 20220615 이후로 지정한 뒤 사용할 Feature를 생성한다.

### Import

In [1]:
import numpy as np
import pandas as pd

# Feature Generate
from haversine import haversine 
from sklearn.decomposition import PCA

# ignore warnings
import warnings ; warnings.filterwarnings('ignore')

### Read Data

In [2]:
train = pd.read_csv('../data/clean/train.csv', parse_dates=['base_date'])
test = pd.read_csv('../data/clean/test.csv', parse_dates=['base_date'])

In [20]:
# train의 기간을 조정한다.
train = train.query('base_date >= "2022-06-15"')

### Feature Generation

In [21]:
feature_train, feature_test = train.copy(), test.copy()

#### $\bullet$ 월

In [22]:
feature_train['base_month'] = train['base_date'].dt.month
feature_test['base_month'] = test['base_date'].dt.month

#### $\bullet$ 좌표계 PCA
- 출발지점의 좌표, 도착지점의 좌표 내 위도간, 경도간 상관관계가 매우 높음을 확인하였다. PCA를 사용해 축소하고자 한다.

In [ ]:
# 방법 1) 위도, 경도끼리 PCA 한다.
lat_pca = PCA(n_components=1, random_state=2022).fit(train[['start_latitude','end_latitude']].values)
long_pca = PCA(n_components=1, random_state=2022).fit(train[['start_longitude','end_longitude']].values)

feature_train['pca_lat'] = lat_pca.transform(train[['start_latitude','end_latitude']].values)
feature_train['pca_long'] = long_pca.transform(train[['start_longitude','end_longitude']].values)
feature_test['pca_lat'] = lat_pca.transform(test[['start_latitude','end_latitude']].values)
feature_test['pca_long'] = long_pca.transform(test[['start_longitude','end_longitude']].values)

In [ ]:
# 방법 2) 출발지점, 도착지점의 좌표를 모두 넣고 PCA의 설명력을 바탕으로 2차원 PCA 한다.
pca = PCA(n_components=2, random_state=2022).fit(train[['start_latitude','start_longitude','end_latitude','end_longitude']].values)

feature_train[['pca1','pca2']] = pca.transform(train[['start_latitude','start_longitude','end_latitude','end_longitude']].values)
feature_test[['pca1','pca2']] = pca.transform(test[['start_latitude','start_longitude','end_latitude','end_longitude']].values)

#### $\bullet$ 하버사인 거리, 방위각
- 하버사인 거리는 구 위의 두 점 사이 거리를 구하는 방법이다.
- 방위각은 한 점의 x축으로부터 다른 점 사이의 각도를 의미하며 방위각 함수(bearing)은 아래의 주소를 참고해 구현하였다.<br>
   [bearing] https://www.movable-type.co.uk/scripts/latlong.html

In [ ]:
feature_train['harversine'] = [haversine((LAT1,LONG1),(LAT2,LONG2), unit='m') 
                               for _,LAT1,LONG1,LAT2,LONG2 in train[['start_latitude','start_longitude','end_latitude','end_longitude']].itertuples()]
feature_test['harversine'] = [haversine((LAT1,LONG1),(LAT2,LONG2), unit='m') 
                              for _,LAT1,LONG1,LAT2,LONG2 in test[['start_latitude','start_longitude','end_latitude','end_longitude']].itertuples()]

In [25]:
def bearing(LAT1, LAT2, LONG1, LONG2):
    lat1, lat2, d_long = map(np.radians, (LAT1, LAT2, LONG2-LONG1))
    y = np.sin(d_long) * np.cos(lat2)
    x = np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(d_long)
    return np.degrees(np.arctan2(y,x))

In [26]:
feature_train['bearing'] = bearing(train['start_latitude'],train['end_latitude'], train['start_longitude'],train['end_longitude'])
feature_test['bearing'] = bearing(test['start_latitude'],test['end_latitude'], test['start_longitude'],test['end_longitude'])

#### $\bullet$ Target Mean
-  Target Mean은 범주형 변수의 범주별 target 평균을 산출한다. 흔히 알고있는 Target Encoding과 동일하나 Feature로서 사용한다.

In [27]:
def targetmean_simple(SIMPLE):
    mean_df = train.groupby(SIMPLE)['target'].agg('mean')
    
    for label in np.unique(test[SIMPLE]):
        if label not in mean_df.index:
            # test에 train에 없는 범주가 나온다면 train.target.mean으로 채운다.
            mean_df.loc[label] = train['target'].mean()
    return mean_df

In [27]:
mean_df = targetmean_simple('road_name')
feature_train['rn_target'] = feature_train['road_name'].map(mean_df)
feature_test['rn_target'] = feature_test['road_name'].map(mean_df)

In [ ]:
mean_df = targetmean_simple('road_rating')
feature_train['rr_target'] = feature_train['road_rating'].map(mean_df)
feature_test['rr_target'] = feature_test['road_rating'].map(mean_df)

In [28]:
mean_df = targetmean_simple('base_hour')
feature_train['hour_target'] = feature_train['base_hour'].map(mean_df)
feature_test['hour_target'] = feature_test['base_hour'].map(mean_df)

In [ ]:
mean_df = targetmean_simple('day_of_week')
feature_train['dow_target'] = feature_train['day_of_week'].map(mean_df)
feature_test['dow_target'] = feature_test['day_of_week'].map(mean_df)

In [29]:
mean_df = targetmean_simple('maximum_speed_limit')
feature_train['msl_target'] = feature_train['maximum_speed_limit'].map(mean_df)
feature_test['msl_target'] = feature_test['maximum_speed_limit'].map(mean_df)

In [30]:
mean_df = targetmean_simple('start_node_label')
feature_train['start_node_target'] = feature_train['start_node_label'].map(mean_df)
feature_test['start_node_target'] = feature_test['start_node_label'].map(mean_df)

In [ ]:
mean_df = targetmean_simple('end_node_label')
feature_train['end_node_target'] = feature_train['end_node_label'].map(mean_df)
feature_test['end_node_target'] = feature_test['end_node_label'].map(mean_df)

In [ ]:
mean_df = targetmean_simple('edge_label')
feature_train['edge_target'] = feature_train['edge_label'].map(mean_df)
feature_test['edge_target'] = feature_test['edge_label'].map(mean_df)

In [31]:
mean_df = targetmean_simple('coor_label')
feature_train['coor_target'] = feature_train['coor_label'].map(mean_df)
feature_test['coor_target'] = feature_test['coor_label'].map(mean_df)

#### $\bullet$ Target Std
- 에서 교통평균속도의 표준편차는 유의한 변수라 밝혔다. 범주형 변수의 범주별 target 표준편차를 산출한다.

In [30]:
def std_simple(SIMPLE):
    std_df = train.groupby(SIMPLE)['target'].agg('std')
    
    for label in np.unique(test[SIMPLE]):
        if label not in std_df.index:
            # test에 train에 없는 범주가 나온다면 train.target.std으로 채운다.
            std_df.loc[label] = train['target'].std()
    return std_df

In [31]:
std_df = std_simple('base_hour')
feature_train['hour_std'] = feature_train['base_hour'].map(std_df)
feature_test['hour_std'] = feature_test['base_hour'].map(std_df)

In [33]:
std_df = std_simple('day_of_week')
feature_train['dow_std'] = feature_train['day_of_week'].map(std_df)
feature_test['dow_std'] = feature_test['day_of_week'].map(std_df)

In [34]:
std_df = std_simple('road_rating')
feature_train['rr_std'] = feature_train['road_rating'].map(std_df)
feature_test['rr_std'] = feature_test['road_rating'].map(std_df)

In [35]:
std_df = std_simple('coor_label')
feature_train['coor_std'] = feature_train['coor_label'].map(std_df)
feature_test['coor_std'] = feature_test['coor_label'].map(std_df)

#### $\bullet$ Frequency
- 범주형 변수의 범주별 빈도는 교통량으로 해석할 수 있다. 그러나, **sampling**된 데이터로 의도와 다를 수 있다. 

In [ ]:
def frequency_simple(SIMPLE):
    fre_df = train.groupby(SIMPLE)['target'].agg('size')
    
    for label in np.unique(test[SIMPLE]):
        if label not in fre_df.index:
            fre_df.loc[label] = fre_df.mean()
    return fre_df

In [ ]:
fre_df = frequency_simple('end_node_label')
feature_train['end_frequency'] = feature_train['end_node_label'].map(fre_df)
feature_test['end_frequency'] = feature_test['end_node_label'].map(fre_df)

In [ ]:
fre_df = frequency_simple('edge_label')
feature_train['edge_frequency'] = feature_train['edge_label'].map(fre_df)
feature_test['edge_frequency'] = feature_test['edge_label'].map(fre_df)

### Save Data

In [ ]:
feature_train.to_csv('../data/feature/feature_train.csv', index=False)
feature_test.to_csv('../data/feature/feature_test.csv', index=False)